In [1]:
chart_annotation_font = {
  'family': 'arial',
  'color':  'black',
  'weight': 'normal',
  'size': 10,
  'style': 'italic'
}

## spend values
spend = np.linspace( 0, 2_000_000, 1000 )

## three spend-revenue functions
revenue1 = 1.2 * spend
revenue2 = 500_000 * np.log1p(spend / 100_000)
revenue3 = 2_000_000 / (1 + np.exp(-0.000005 * (spend - 1_000_000))) 

curves = [ 
            ( revenue1, "Revenue = 0.9 × Spend", "Spend-Revenue Curve 1 (Linear)", 'dodgerblue' ),
            ( revenue2, "Revenue = 500k × ln(1 + Spend / 100k)", "Spend-Revenue Curve 2 (Log)", 'maroon' ),
            ( revenue3, "Revenue = 2M / (1 + e^{-0.000005 × (Spend - 1M)})", "Spend-Revenue Curve 3 (Logit)", 'mediumvioletred' )
]
rho = 1.2

NameError: name 'np' is not defined

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as mtick

## create the plot
fig, axs = plt.subplots( 3, 1, dpi=300, figsize=( 12, 8 ) )
fig.patch.set_facecolor( 'xkcd:navy' )

for ax, (revenue, eq, title, color) in zip( axs, curves ):
    marginal_roas = np.gradient( revenue, spend )
    ## seed the index with the last value 
    ## eg., linear model where all gradients are the same
    idx = len(spend) - 1
    ## if the gradients are not all 0 or close to it, select the index
    if not np.all( np.isclose( marginal_roas, rho, rtol=1e-4 ) ):
        idx = np.argmin( np.abs( marginal_roas - rho ) )
    optimal_spend = spend[ idx ]
    optimal_revenue = revenue[ idx ]
    ## mark the optimal spend amount
    ax.axvline( x=optimal_spend, color='white', linestyle='--', linewidth=2, alpha=0.8 )
    ## annotate
    ax.text( optimal_spend-250_000, optimal_revenue+250_000, 
        f'Optimal Spend: ${optimal_spend:,.0f}', 
        fontsize=9, color='black', 
        verticalalignment='bottom', horizontalalignment='left' )
    ## red marker
    ax.plot( optimal_spend, optimal_revenue, 'o', color='red', markersize=8 )
    ax.plot( spend, revenue, linewidth=4, color=color )
    ax.set_title( title, color='white', fontsize=12 )
    ax.set_xlabel( "Ad Spend ($)", color='white' )
    ax.set_ylabel( "Revenue ($)", color='white' )
    ax.tick_params( axis='x', colors='white' )
    ax.tick_params( axis='y', colors='white' )
    ax.set_xlim( 0, 2_200_000 )
    ax.set_ylim( 0, 3_000_000 )
    ax.xaxis.set_major_locator( mtick.MultipleLocator( 500_000 ) )
    ax.yaxis.set_major_locator( mtick.MultipleLocator( 500_000 ) )
    ax.xaxis.set_major_formatter( mtick.StrMethodFormatter('${x:,.0f}' ) )
    ax.yaxis.set_major_formatter( mtick.StrMethodFormatter('${x:,.0f}' ) )
    ax.text( 0.05, 0.95, eq, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', color='black' )
    ax.grid( True, which='both', color='xkcd:charcoal', linewidth=0.8, linestyle=':', alpha=.3 )

axs[ 2 ].set_xlabel(
    "Chart created by Eric Benjamin Seufert, Mobile Dev Memo",
    fontdict=chart_annotation_font,
    labelpad=20,
    color="white"
)

fig.suptitle( "Arbitrary Spend-Revenue Curves", weight='bold', color='white', fontsize=22)
plt.tight_layout()
plt.show()


In [ ]:

from scipy.optimize import minimize
import pandas as pd

# Define revenue functions
def revenue1(s): return 1.2 * s
def revenue2(s): return 500_000 * np.log1p(s / 100_000)
def revenue3(s): return 2_000_000 / (1 + np.exp(-0.000005 * (s - 1_000_000)))

# Pack functions
revenue_functions = [revenue1, revenue2, revenue3]
N = len(revenue_functions)

# Objective function: maximize ROAS = total revenue / total spend
# Use Charnes-Cooper transformation: x_i = s_i * t
def transformed_objective(x_t):
    x = x_t[:N]
    t = x_t[-1]
    if t <= 0:
        return 1e9  # Penalize non-positive t
    revenue_sum = sum(revenue_functions[i](x[i]/t) for i in range(N))
    return -revenue_sum  # Negative for maximization

# Constraints:
# 1. sum(x_i) = 1
# 2. ROAS_i >= rho: Revenue_i / Spend_i >= rho => Revenue_i(x_i/t)/(x_i/t) >= rho
# 3. x_i >= 0, t > 0
constraints = [
    {'type': 'eq', 'fun': lambda x_t: np.sum(x_t[:N]) - 1},  # sum x_i = 1
]

for i in range(N):
    constraints.append({
        'type': 'ineq',
        'fun': lambda x_t, i=i: (revenue_functions[i](x_t[i]/x_t[-1]) / (x_t[i]/x_t[-1])) - rho
    })

# Bounds: x_i >= 0, t > 0
bounds = [(0, None)] * N + [(1e-6, None)]  # t must be > 0

# Initial guess
x0 = [1/N] * N + [1]

# Optimize
result = minimize(transformed_objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)

# Extract solution
x_opt = result.x[:N]
t_opt = result.x[-1]
s_opt = x_opt / t_opt
revenue_opt = [revenue_functions[i](s_opt[i]) for i in range(N)]
roas_opt = [revenue_opt[i] / s_opt[i] for i in range(N)]

# Display
import ace_tools as tools; tools.display_dataframe_to_user(name="Optimization Results", dataframe=pd.DataFrame({
    'Channel': [f'Channel {i+1}' for i in range(N)],
    'Spend ($)': s_opt,
    'Revenue ($)': revenue_opt,
    'ROAS': roas_opt
}))
